# Identifying Private Personally Identifiable Information

Our next task is to combine all our sources of data in order to determine which incident cards are releasable to the public. In particular we want to know which cards are:

* Releasable
* Not Releasable
* Indeterminate

The Indeterminate category is very important. Our automated approach will not produce an answer for every situation. So it is important that we allow these indeterminate situations to be known and reported to people who can check for PII by hand.

**NOTE**: It is important that we only release records for which the individual is **proved** by supporting evidence to be over 18 at the time. If we cannot match a name with supporting evidence, then it will be redacted. Either additional dataset may be used to prove their age, but at least one is required for release.

Before we get into the particular records that make this possible, let's perform some package imports.

In [2]:
import pandas as pd
from datetime import datetime, timedelta as delta

## Determination of Staff or Incarceree

The incident cards contain records about internment camp staff, as well as incarcerees. The staff cards must be redacted from the dataset. Lucky for us, that data is found in the incident card data itself and does not require a lookup in another source.

In [3]:
data_Card = pd.read_csv( "Datasets/Cards_Box9.csv" )
data_Card.head(3)

,Box #,Not Inmate,Last Name,First Name,Other Name,Date,Year
0,Box9-0788.jpg,NaN,Enomoto,Masanobu,NaN,7/21/42,1942
1,Box9-0692.jpg,NaN,Ebesu,Kikumatsu,NaN,7/24/42,1942
2,Box9-0642.jpg,NaN,Doi,Satomi,NaN,8/6/42,1942


You can see in the "Not Inmate" column, that staff are marked with a "Y" value, because they were not considered "inmates". All of the incarceree rows show "NaN" values in that column, which stands for "not a number". This is how Python describes a missing value, i.e. no data. Pandas has a useful function for testing if a value is missing or not called `isna(value)`. We use that function in the code below to count the number of incarceree and staff cards.

In [3]:
incarcerees = 0
staff = 0
for i in range( 0, data_Card.shape[0] ):
    if pd.isna( data_Card.iloc[ i, 1 ] ):
        incarcerees = incarcerees+1
    else:
        staff = staff+1
print("We found %5s incident cards about incarcerees and %5s about staff." % (incarcerees, staff))

We found    96 incident cards about incarcerees and    17 about staff.


## Determination of Juvenile Records

We have two important sources of age data that we can consult, in the WRA Form 26 dataset and the FAR dataset. Let have a peek to remind ourselves what those datasets look like.

In [4]:
data_Form26 = pd.read_csv("Datasets/WRAForm26.csv")
data_Form26.tail(3)

,LastName,FirstName,BirthYear
109189,ZORIKI,MIKE,1922
109190,ZORIKI,JUDY,1942
109191,ZUICHO,FUMIO,1887


In [5]:
data_FAR = pd.read_csv("Datasets/TuleLake_FAR_ALL_FINAL4.csv")
data_FAR.tail(3)

,OriginalOrder,LastName,FirstName,OtherNames,DOB,BirthYear
28876,30320,Yutani,Yoshie,William,5/11/1940,1940
28877,30321,Yuzuki,Masaichi,NaN,12/15/1893,1893
28878,30322,Yuzuki,Minoru,NaN,8/29/1922,1922


So it seems that the Form 26 data has the year of birth, while the FAR dataset include both the birth year and the date of birth (DOB). By looking at the maximum value of the birth year in FAR, we can see that some incarcerees were infants at the time.

In [6]:
data_FAR[ 'BirthYear' ].max()

1945

## Cross-Referencing Datasets: Lookup Functions and Indexing

Given any particular card data from the incident card dataset, we need to determine if the individual in question was under the age of 18 at the time. The card contains the date of the incident, but in order to determine the birthdate or birth year of an individual, we need to consult our other data sources. In order to do that, we have to use the first and last names to look them up in the other datasets.

One easy way to approach algorithms is to first map out the same process as if a human were going to perform it. Then you have a clear idea of the steps that need to be implemented ni your code. We have three data source that need to be consulted, one identifies staff records and the second two identify those who were minors at the time of an incident. If we make a chart of the process a human might follow, it would look like this:

<img src="Pics/flowchart.png"/>

If you are not especially familiar with a flow chart diagram, the cylinders represent data storage, the diamonds represent choices, and the smooth rectangles represent a process or procedure. The rectangles with curves on the bottom represent output and are meant to resemble paper. So now you can see the several decision points and procedures that make up the overall procedure that a human or machine must follow to make the PII decision for all our incident cards.

This chart shows a repeated outer loop, using the assigned variable 'i' to iterate through all of the cards. This kind of detail might be omited in a procedure for humans, since we just assume that the same steps are applied to every card. However, since we want to map the overall task, we make it explicit that the procedure must process all of the cards.

### **Student Activity**: Counting Decisions and Processes

Look at the flowchart above. Assume that we have 250 incident cards with a random mix of staff, juveniles, and adults. Answer these questions:

* What is the maximum number of decisions a person might have to make to process all of the cards? 
* What is the minimum number of decisions a person would have to make?
* What is the minimum number of name lookups a person might have to do?
* What is the minimum number of name lookups a person might have to do?
* If each name lookup takes a human one minute and each decision takes 15 seconds, what is the maximum human processing time?

## Functional Decomposition

Functionally speaking, a computer must perform exactly the same steps as a human to complete this work. They follow the same procedure in the flowchart. There are many, many ways to program this procedure in Python and some are more efficient and easier to understand than others. Generally a good first approach is to break the larger problem down into functional pieces and in computer science this practice is called **functional decomposition**. With our PII determination, the functional decomposition has already happened, in the form of our flowchart. We have identified two smaller "name lookup" processes within the larger procedure. One is to look up a name in the FAR registry and the other is to look up a name in WRA Form 26 records. Let's look more deeply at the "name lookup" process.

In the paper world a ordinary person would perform name lookups by looking at each record, in Form 26 for example, in turn until they either found a match with the incident card or reached the last record without a match. They would compare first and last names between the incident card and the paper record. If a last name doesn't match, then they can move on without also checking the first name. What we see here is that the "name lookup" procedure could have a flowchart as well, including a loop that checks each FAR record in turn against a given last name and then against a given first name. This is simple enough to implement in Pandas code.

In [7]:
def lookup_Form26(last, first):
    """Looks up a name in Form 26, looking at each record in turn."""
    last = last.lower()  # We are going to standardize on lower case matches
    first = first.lower()
    values = data_Form26.values  # This gives us a Python array of the FAR records
    for i in range(0, len(values)):  # loop through FAR records; array indices start at 0
        F26_lastname = values[i][0].lower()  # Last name is column 0
        if last == F26_lastname:
            F26_firstname = values[i][1].lower()  # First name is column 1
            if first == F26_firstname:
                return values[i]  # if both names match, return this record

In [8]:
def test_lookup_Form26():
    """Testing lookup function"""
    first_name = 'Fumio'
    last_name = 'Zuicho'
    result = lookup_Form26(last_name, first_name)  # Result will be None if no match..
    print('I got: '+ str(result))
    
test_lookup_Form26()

I got: ['ZUICHO' 'FUMIO' 1887]


## Optimization
If this name lookup process seems straightforward to you, recall that the FAR dataset has 28879 records and the WRA Form 26 dataset has 109,192 records. If an archivist consulted each record in turn, it would take a very long time to find a name. In practice an archivist would rarely need to look at each record in turn. They would use a time saving device, either alphabetized records or an alphabetized index of the records. A modern computer user performing the same task against records in a spreadsheet would use the sort feature to alphabetize the records by last name. Now that we've identified the smaller functional parts of our overall task and written our first lookup implementation, it is time to consider efficiency.

Let's first figure out how slow our first lookup implementation is..

In [9]:
import timeit
secs = timeit.timeit(test_lookup_Form26, number=10)
print('Average time is %5s seconds.' % secs)

I got: ['ZUICHO' 'FUMIO' 1887]
I got: ['ZUICHO' 'FUMIO' 1887]
I got: ['ZUICHO' 'FUMIO' 1887]
I got: ['ZUICHO' 'FUMIO' 1887]
I got: ['ZUICHO' 'FUMIO' 1887]
I got: ['ZUICHO' 'FUMIO' 1887]
I got: ['ZUICHO' 'FUMIO' 1887]
I got: ['ZUICHO' 'FUMIO' 1887]
I got: ['ZUICHO' 'FUMIO' 1887]
I got: ['ZUICHO' 'FUMIO' 1887]
Average time is 1.0059756562113762 seconds.


Note that we are looking up the very last record in the largest dataset of 109,192 records. So a few seconds is still much faster than a person could perform this work. In fact a few seconds might seem fast enough to us. But remember that ever a person would use some indexing techniques to increase their speed. Similar techniques are readily available in Pandas for our use in our lookup function. Also, remember that we have to perform this lookup up to 250 times.

### Indexing Last Names in Form 26

Now let's look at how we build an index in Pandas, starting with the last names.

In [10]:
g_FAR = data_FAR.groupby(data_FAR['LastName'].str.lower()) # g.groups is now a Python dictionary
far_list = list( g_FAR.groups )

print( "First 12 LastName KEYS, out of a total of: ", len( far_list ), "\n")
print( far_list[0:12] )  # the colon or ':' is part of Python's "slice" syntax

First 12 LastName KEYS, out of a total of:  2947 

['abe', 'abey', 'abiko', 'abo', 'aburamen', 'adachi', 'adachi (kono)', 'aida', 'aiura', 'aiura (ajimi)', 'aizawa', 'ajima']


In [11]:
g_dataForm26 = data_Form26.groupby(data_Form26['LastName'].str.lower()) # Note that we have made the last name lowercase in the groupby index.
form26_list = list( g_dataForm26.groups )

print( "First 12 LastName KEYS, out of a total of: ", len( form26_list ), '\n')
print( form26_list[0:12] )

First 12 LastName KEYS, out of a total of:  6483 

['aanagawa', 'aawato', 'abbey', 'abe', 'abey', 'abiko', 'abo', 'aburamen', 'aburano', 'achiu', 'acki', 'adachi']


In the two examples above we have instructed our Pandas Data Frame to create groups of records using the last name column. Each group of records is
then available separately in a "groups" dictionary property, with the last name as the retreival key. To verify that we have the data we expect, we
printed the first twelve keys in the dictionary. If we pick a key from the list above, we can see what values are stored under that key.

In [12]:
print(list(g_dataForm26.groups.get('abbey')))  # Uses the get() function to retrieve the array indices of ABBEY records

[2, 3, 4, 5]


Looking at the array of integers that is stored for the family name "ABBEY", we can see that each value in the list is an "index" within
our Pandas data frame. Indexing creates another way to access the data frame using the family name. The index only stores the index numbers, or locations, 
where matching records are found in the original data frame. We still have one dataset array, but now have a new index into that array. This same
principle is used in modern database systems and in old fashioned card catalogs.

Let's retreive the values associated with this family name.

In [13]:
print(g_dataForm26.get_group('abbey'))  # get_group() returns a data frame for the group.

  LastName FirstName  BirthYear
2    ABBEY       ROY       1905
3    ABBEY    HARUKO       1916
4    ABBEY     LEONA       1937
5    ABBEY     AYAKO       1941


In [14]:
data_Form26.head(7)  # Now let's look at the beginning of the original data frame.

,LastName,FirstName,BirthYear
0,AANAGAWA,MARY,1927
1,AAWATO,HISAKICH,1889
2,ABBEY,ROY,1905
3,ABBEY,HARUKO,1916
4,ABBEY,LEONA,1937
5,ABBEY,AYAKO,1941
6,ABE,YUMOSHIN,1864


We can see that our groups are storing the index of each occurance of "ABBEY". The family names were lowercased as they were being indexed, but this did not change the value in our data frame.

Now let's rewrite our lookup function to take advantage of the indexed family names.

In [15]:
def lookup_Form26_indexed(last, first):
    """Looks up a name in Form 26, starting with the last name index."""
    last = last.lower()  # We are going to standardize on lower case matches
    first = first.lower()
    if g_dataForm26.groups.get(last) is None:
        return None  # No match on last name
    for i in list(g_dataForm26.groups.get(last)):
        val = data_Form26.values[i]
        if first == val[1].lower():
            return val  # if both names match, return this record

Now let's test the new lookup function and time its performance.

In [16]:
def test_lookup_Form26_indexed():
    """Testing lookup function"""
    first_name = 'Fumio'
    last_name = 'Zuicho'
    result = lookup_Form26_indexed(last_name, first_name)  # Result will be None if no match..
    print('I got: '+ str(result))

import timeit
secs = timeit.timeit(test_lookup_Form26, number=10)
print('Average time is %5s seconds.' % secs)

I got: ['ZUICHO' 'FUMIO' 1887]
I got: ['ZUICHO' 'FUMIO' 1887]
I got: ['ZUICHO' 'FUMIO' 1887]
I got: ['ZUICHO' 'FUMIO' 1887]
I got: ['ZUICHO' 'FUMIO' 1887]
I got: ['ZUICHO' 'FUMIO' 1887]
I got: ['ZUICHO' 'FUMIO' 1887]
I got: ['ZUICHO' 'FUMIO' 1887]
I got: ['ZUICHO' 'FUMIO' 1887]
I got: ['ZUICHO' 'FUMIO' 1887]
Average time is 0.7969476748257875 seconds.


The performance improvement will vary from system to system, but it should be from twice to almost ten times faster with an index.

## Complexity of Matching Personal Names

Up until now we have pretended that our card dataset and our lookup datasets always include a first and last name. In fact,
we know that Form 26 is like this, but the FAR dataset includes last names, first names, and other names. In many cases either
the first name or the other name is blank. 

Given the name data on an incident card, which is often also partially blank, we need to do the best job we can to match the lookup records. A matched records will have the same last name (lowercase) and also either the same first name or the same other name. With respect to Form 26, we will also look for the card's other name
in the Form 26 first name column. This reflects reality, in which a person's name may change over time. Someone may become more and more known by a name which was rarely used in the past. An incident card might record their current common name and not the first name that was recorded on Form 26.

So now our lookup function must include the additional other name parameter and it's matching logic must be expanded to include the possible match of Form 26 first names with incident card other names.

In [17]:
def lookup_Form26_indexed_complex(last, first, other):  # note the additional "other" parameter
    """Looks up a name in Form 26, starting with the last name index."""
    last = last.lower()
    if g_dataForm26.groups.get(last) is None:
        return None  # No match on last name
    for i in list(g_dataForm26.groups.get(last)):
        val = data_Form26.values[i]
        if first is not None and first.lower() == val[1].lower():
            return val  # if both names match, return this record
        elif other is not None and other.lower() == val[1].lower():
            return val  # card other name matches form 26 first name
    return None  # if we reach the end of the family group w/o finding a matching value.

In [18]:
result = lookup_Form26_indexed_complex('Antoku', None, 'Dorothy')  # This example taken from card data.
print('1 got: '+ str(result))
   
result = lookup_Form26_indexed_complex('Antoku', 'Dorothy', None)  # This example taken from card data.
print('2 got: '+ str(result))

result = lookup_Form26_indexed_complex('Antoku', None, 'Charles')  # This example taken from card data.
print('3 got: '+ str(result))

1 got: ['ANTOKU' 'DOROTHY' 1920]
2 got: ['ANTOKU' 'DOROTHY' 1920]
3 got: None


If you look at the results here, you can see that we are able to match Dorothy with her Form 26 data and that she was born in 1920. Our perforance timing script took the average of ten calls to the function, hence the duplicate output. However, after that we attempted to match "Charles Antoku" with his Form 26 record, but found nothing. Let's look at this further by printing out all of the Antoku records.

In [19]:
for i in list(g_dataForm26.groups.get('antoku')):
    print(data_Form26.values[i])

['ANTOKU' 'YETARO' 1878]
['ANTOKU' 'YOSHITAR' 1884]
['ANTOKU' 'SHIMO' 1893]
['ANTOKU' 'MUMENO' 1895]
['ANTOKU' 'SAKAYE' 1904]
['ANTOKU' 'TORIYE' 1917]
['ANTOKU' 'TED' 1919]
['ANTOKU' 'DOROTHY' 1920]
['ANTOKU' 'CHARLIE' 1923]
['ANTOKU' 'GEORGE' 1924]
['ANTOKU' 'MARY' 1927]
['ANTOKU' 'EDITH' 1930]


### **Student Activity**: Complex Names

Analyze the situation of matching "Charles Antoku" with Form 26 data.

**Questions:**
* Why was there no match?
* Without this match, could Charles' incident card still be released? Why or why not?
* Do we have to fix this mis-match in order to protect Charlie's private information?

## Finished Lookup Functions

Below we give a shorter name to our Form 26 lookup function and we create a similar lookup function for the FAR dataset. Note that the FAR dataset has values that are not strings for blank name data. These are in the dataset as "nan", which means "not a number". Because Pandas was developed in large part for crunching numbers, missing values are encoded in a numeric way. For this reason we test for missing FAR data by testing if the value is a string or not, using Python's `isinstance()` function.

In [20]:
lookup_form26 = lookup_Form26_indexed_complex  # this creates a new function based on the one with the long name

def lookup_far(last, first, other):
    """Looks up a name in FAR, starting with the last name index."""
    last = last.lower()
    if g_FAR.groups.get(last) is None:
        return None  # No match on last name
    for i in list(g_FAR.groups.get(last)):
        val = data_FAR.values[i]
        if first is not None and isinstance(val[2], str) and first.lower() == val[2].lower():
            return val  # if both names match, return this record
        elif other is not None and isinstance(val[3], str) and other.lower() == val[3].lower():
            return val  # card other name matches form 26 first name
    return None  # if we reach the end of the family group w/o finding a matching value.

Let's test this before we move on.

In [21]:
result = lookup_far('Abe', 'Junko', 'June')  # This example taken from card data.
print('1 got: '+ str(result))
   
result = lookup_far('Abe', None, 'June')
print('2 got: '+ str(result))

result = lookup_far('Abe', 'Junko', None)
print('3 got: '+ str(result))

1 got: [26 'Abe' 'Junko' 'June' '6/17/1927' 1927]
2 got: [26 'Abe' 'Junko' 'June' '6/17/1927' 1927]
3 got: [26 'Abe' 'Junko' 'June' '6/17/1927' 1927]


## To be continued

In the [next part](PII_Algorithm2.ipynb) we will put our look up functions to use in making incident card release decisions.